# **ECONOMETRICS NB** 
## Thomas de Portzamparc 27 october 2025

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import statsmodels.api as sm

In [2]:
def load_data(file_path):
    data = pd.read_csv(file_path, index_col=0, parse_dates=True)
    return data
file_path = "../data/market_data_full_D.csv"
df = load_data(file_path)

print(df.info())
df.head()
# # Scatter Matrix -> commented out because very long to run
# scatter_matrix(df[['Y', 'X', 'INF', 'TS', 'DI', 'DP', 'DM', 'DC', 'DS', 'DT']], figsize=(12, 8))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 704 entries, 2023-01-04 to 2025-10-23
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   AAPL        704 non-null    float64
 1   SPY         704 non-null    float64
 2   QQQ         704 non-null    float64
 3   XLK         704 non-null    float64
 4   TLT         704 non-null    float64
 5   VIXY        704 non-null    float64
 6   GLD         704 non-null    float64
 7   UUP         704 non-null    float64
 8   INF         704 non-null    float64
 9   DP          704 non-null    float64
 10  DM          704 non-null    float64
 11  DC          704 non-null    float64
 12  DS          704 non-null    float64
 13  TS          704 non-null    float64
 14  DT          704 non-null    float64
 15  RF_daily    704 non-null    float64
 16  RF_monthly  704 non-null    float64
dtypes: float64(17)
memory usage: 99.0 KB
None


,AAPL,SPY,QQQ,XLK,TLT,VIXY,GLD,UUP,INF,DP,DM,DC,DS,TS,DT,RF_daily,RF_monthly
2023-01-04,0.989930,0.740192,0.479020,0.291687,1.380142,10.99,0.925313,-0.356253,0.010948,-0.019322,-0.023620,0.007135,0.04,-3.650645,-0.003548,0.017661,0.371534
2023-01-05,-1.081866,-1.155840,-1.585348,-1.985603,0.388086,11.08,-1.223934,0.817492,0.010947,-0.019326,-0.023626,0.007135,0.01,-3.704194,-0.053548,0.017674,0.371817
2023-01-06,3.559039,2.259821,2.689257,2.895688,1.832625,10.81,1.836087,-1.157111,0.010946,-0.019330,-0.023631,0.007134,-0.02,-3.647742,0.056452,0.017688,0.372101
2023-01-09,0.477401,-0.028354,0.645451,1.139954,0.507363,10.88,0.221414,-0.664693,0.010942,-0.019341,-0.023648,0.007133,-0.01,-3.678387,-0.010215,0.017728,0.372953
2023-01-10,0.429251,0.693644,0.894490,0.616359,-1.674058,10.42,0.366983,0.072072,0.010941,-0.019345,-0.023654,0.007132,-0.01,-3.671935,0.006452,0.017742,0.373237


In [ ]:
def estimate_atm_and_skew(returns):
    """
    Estimate historical ATM vol and leverage skew from returns.
    """
    sigma_hist = returns.std() * np.sqrt(252)
    leverage_corr = np.corrcoef(returns, returns**2)[0,1]
    rho = np.clip(-abs(leverage_corr), -0.9, -0.05)  # skew négative pour actions
    return sigma_hist, rho


def ssvi_total_variance(k, theta, rho, phi):
    return 0.5 * theta * (1 + rho * phi * k + np.sqrt((phi * k + rho)**2 + 1 - rho**2))

def ssvi_total_variance(k, theta, rho, phi):
    return 0.5 * theta * (1 + rho * phi * k + np.sqrt((phi * k + rho)**2 + 1 - rho**2))

def build_smile_from_hist(S0, sigma_hist, rho, T, phi=0.2, moneyness=np.linspace(0.7, 1.3, 31)):
    theta = (sigma_hist**2) * T
    phi = phi / np.sqrt(theta)
    F0 = S0
    K = F0 * moneyness
    k = np.log(K / F0)
    w = ssvi_total_variance(k, theta, rho, phi)
    iv = np.sqrt(np.maximum(w, 1e-12) / T)
    smile = pd.DataFrame({"K": K, "log_moneyness": k, "IV": iv})
    param = {"theta": theta, "rho": rho, "phi": phi, "T": T}
    return K, iv, smile, param

   

def build_vol_surface(S0, sigma_hist, rho, maturities_days=[10, 30, 60, 90, 180, 365]):
    moneyness = np.linspace(0.7, 1.3, 41)
    T_list = np.array(maturities_days) / 365
    K_grid, T_grid = np.meshgrid(S0 * moneyness, T_list)
    IV_grid = np.zeros_like(K_grid)

    for i, T in enumerate(T_list):
        K, iv, smile, param = build_smile_from_hist(S0, sigma_hist, rho, T, phi=0.25, moneyness=moneyness)
        IV_grid[i, :] = iv

    return K_grid, T_grid, IV_grid

In [11]:
import plotly.graph_objects as go
import numpy as np

# --- Surface Plotly ---
fig = go.Figure(
    data=[go.Surface(
        x=K_grid,
        y=T_grid * 365,  # jours
        z=IV_grid,
        colorscale='Viridis',
        showscale=True,
        contours = {
            "z": {"show": True, "usecolormap": True, "highlightcolor": "limegreen", "project_z": True}
        }
    )]
)

fig.update_layout(
    title=f"AAPL Synthetic Volatility Surface (SSVI)<br>σ_ATM={sigma_hist:.2%}, ρ={rho:.2f}",
    scene=dict(
        xaxis_title="Strike",
        yaxis_title="Maturity (days)",
        zaxis_title="Implied Volatility",
        xaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.3)'),
        yaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.3)'),
        zaxis=dict(showgrid=True, gridcolor='rgba(200,200,200,0.3)')
    ),
    margin=dict(l=50, r=50, b=50, t=80),
    template="plotly_dark"
)

fig.show()
